In [2]:
# let's start by loading the environment variables

import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

assert key, "Please set the AZURE_OPENAI_API_KEY environment variable"
assert endpoint, "Please set the AZURE_OPENAI_ENDPOINT environment variable"
assert deployment_name, "Please set the DEPLOYMENT_NAME environment variable"
assert api_version, "Please set the AZURE_OPENAI_API_VERSION environment variable"

In [3]:
# Example 1: Vanilla chat experience

from openai import AzureOpenAI

client = AzureOpenAI(api_key=key, api_version=api_version, azure_endpoint=endpoint)  # type: ignore (asserted already)

response = client.chat.completions.create(
    model=deployment_name,  # type: ignore (asserted already)
    messages=[
        {
            "role": "system",
            "content": "You are an interesting know-it-all person.",
        },
        {"role": "user", "content": "Tell me one interesting fact."},
    ],
)

print(response.choices[0].message.content)

One interesting fact is that honey never spoils. Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible. This is due to its low moisture content and acidic pH, which create an inhospitable environment for bacteria and microorganisms.


In [4]:
# Example 2: Introducing LangChain

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    api_version=api_version,  # type: ignore
    azure_deployment=deployment_name,
)


sys_msg = SystemMessage(content="You are an interesting know-it-all person.")
human_msg = HumanMessage(
    content="Tell me one interesting fact",
)
# introduce randomness in the response
response = model.invoke([sys_msg, human_msg], temperature=0.9, top_p=0.9, frequency_penalty=0.5, presence_penalty=0.6)

response.content

"Did you know that honey never spoils? Archaeologists have discovered pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible! This is due to honey's unique chemical composition, low moisture content, and acidic pH, which create an inhospitable environment for bacteria and microorganisms."

In [7]:
# Example 3: Log Analysis

from pathlib import Path

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    api_version=api_version,  # type: ignore
    azure_deployment=deployment_name,
)

log = Path("demo.log").read_text()

sys_msg = SystemMessage(content="You are an expert software engineer.")
human_msg = HumanMessage(
    content=f"Analyze the following log file and create a consie one-paragraph-long summary of it, highlighting anomalies:\n\n<logs>\n\n{log}\n\n</logs>",
)
# introduce randomness in the response
response = model.invoke([sys_msg, human_msg], temperature=0.3)

print(response.content)

The log file indicates that the server started successfully and handled multiple requests without issues initially. However, it encountered significant anomalies related to database connectivity, including a timeout while executing a user query, followed by a connection refusal error, which led to a service degradation warning. An automatic database failover was triggered, switching to a secondary instance, and connectivity was re-established shortly thereafter. Additionally, there was another error indicating a connection reset by peer, accompanied by a warning of intermittent database connectivity issues. Despite these challenges, the server continued to process requests and completed scheduled tasks without further incident.
